In [1]:
import os
import asyncio
import aiohttp
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from dotenv import load_dotenv
from ratelimit import limits, sleep_and_retry

# Load environment variables
load_dotenv()
api_key = os.getenv('api')

# Rate limit parameters
RATE_LIMIT = 100  # 100 requests per minute
TIME_WINDOW = 60  # 60 seconds

# Database connection
engine = create_engine('mssql+pyodbc://Revision-PC\\SQLEXPRESS/RiotDataDB?driver=ODBC+Driver+17+for+SQL+Server')

# Decorator to handle rate limiting
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_WINDOW)
async def fetch_puuid(session, summoner_id):
    api_puuid = f"https://br1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={api_key}"
    async with session.get(api_puuid) as response:
        response.raise_for_status()
        puuid_json = await response.json()
        return puuid_json.get('puuid', None)

async def fetch_all_puuids(session, summoner_ids):
    tasks = [fetch_puuid(session, summoner_id) for summoner_id in summoner_ids]
    return await asyncio.gather(*tasks)

async def main():
    # Get the desired league from user input
    liga = input("Insira a liga desejada: ").upper()
    api_liga = f"https://br1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{liga}/I?api_key={api_key}"
    
    # Fetch summoner IDs from the league
    async with aiohttp.ClientSession() as session:
        async with session.get(api_liga) as liga_resposta:
            liga_resposta.raise_for_status()
            liga_json = await liga_resposta.json()

    summoner_ids = [entry['summonerId'] for entry in liga_json]
    df_liga = pd.DataFrame(summoner_ids, columns=['summonerId'])

    # Fetch PUUIDs asynchronously
    async with aiohttp.ClientSession() as session:
        puuids = await fetch_all_puuids(session, tqdm(df_liga['summonerId'], desc="Fetching PUUIDs", unit="summonerId"))
    
    # Add the PUUIDs to the DataFrame
    df_liga['puuid'] = puuids

    # Save the DataFrame to SQL Server
    df_liga.to_sql('SummID_PUUID', con=engine, if_exists='replace', index=False)
    print("Data successfully saved to the database.")

if __name__ == "__main__":
    asyncio.run(main())


Fetching PUUIDs: 100%|██████████| 205/205 [03:36<00:00,  1.05s/summonerId]

                                          summonerId  \
0    _fnScKadKNbKQovSJDPLDGY5IlzhuvWmJANbIscjf3RnUtc   
1    LmA2XeOkbKnitfif8apk8mUipTgoJxim41X60Rq5e7l5JF8   
2    tppRxyf3m5W7nP5Qkbz83ZLzR_5yKo7V5hkujXcr1ES8JH8   
3    yST71AQ3hTsngdOMubZCAiKY1kvXvzHh9aHmaqd1_SvXItQ   
4    Q1DDP5_3qDeD2mk6vWPTTpNYDn4MACabilz9PxeDycv7A1k   
..                                               ...   
200  6HBFGoeLtQkNgtT5eLTFJ-OyJnpjF4HxJS3nEYQRUef6x9E   
201  RRq5QmAO4z8j5pwX3O5_Qu-YfXT_osLnxZ9hGg1KcnYx_Wk   
202  nEjWtjwOrTTKL9mDypFuO3pEt41IGgUifCD-ii8kMtoS5LE   
203  UPWe3g8qgvHcWyet2SoDWRZBWO7I4rfNA0HgBHztUuNXujU   
204  LJwWUJgQGYKixSmchjAUPVsvBMthb086hARmn41jdlwGOCA   

                                                 puuid  
0    bde2Ss0TBM-PAUkbP3gBxpELAM5ho6GMC5jgCvic5mQ4Tr...  
1    3w3e2I0ItmSwKUtB87ASHnF7UKWvcF-XlxOWzuVMBub_bv...  
2    iWBUKiraPtA1pTyAs0amw0CmAZYWmoplUiLugG2HgC_YPh...  
3    Th1-8DOc-nXyPQUBQf9M4uqwtlCFu6BWpyudv5TQG7k7l3...  
4    RhlWOrZaBjtsn6g0m-nYnigDVRsa2Nl5EVXvT